# 1. Import the required libraries

In [ ]:
# Standard code libraries
import os
import matplotlib.pyplot as plt

# Custom code libraries from ReSurfEMG
from resurfemg.data_connector.config import Config
from resurfemg.data_connector import file_discovery
from resurfemg.pipelines import ipy_widgets
from resurfemg.data_connector.tmsisdk_lite import Poly5Reader
from resurfemg.data_connector.data_classes import (
VentilatorDataGroup, EmgDataGroup)

%matplotlib widget

## 2. Load the ventilator and sEMG data

In [ ]:
# Identify all recordings available for the selected patient/measurement_date

# First find the patients
config = Config()
base_path = config.get_directory('patient_data')
patient_paths = file_discovery.find_folders(
    base_path,
    folder_levels=['patient'])
patient = list(patient_paths['patient'].values)[0]

# Then find the files for the selected patients:
folder_levels = ['date', 'measurement']
patient_path = os.path.join(base_path, patient)

emg_files = file_discovery.find_files(
    base_path=patient_path,
    file_name_regex='EMG_recording',
    extension_regex='poly5',
    folder_levels=folder_levels)

folder_levels = ['date', 'measurement']
vent_files = file_discovery.find_files(
    base_path=patient_path,
    file_name_regex='Draeger_recording',
    extension_regex='poly5',
    folder_levels=folder_levels)

button_list = ipy_widgets.file_select(
    emg_files,
    folder_levels=folder_levels[:-1],
    default_value_select=None,
    default_idx_select=[1, 6])

In [ ]:
emg_file_chosen = os.path.join(
    patient_path, *[btn.value for btn in button_list], 'EMG_recording.Poly5')
vent_file_chosen = os.path.join(
    patient_path, *[btn.value for btn in button_list], 'Draeger_recording.Poly5')

print("The chosen files are:\n", emg_file_chosen, '\n', vent_file_chosen)

In [ ]:
# Load the EMG and ventilator data recordings from the selected folders.
data_emg = Poly5Reader(emg_file_chosen)
data_vent = Poly5Reader(vent_file_chosen)
data_emg_samples = data_emg.samples[:data_emg.num_samples]
fs_emg = data_emg.sample_rate
data_vent_samples = data_vent.samples[:data_vent.num_samples]
fs_vent = data_vent.sample_rate

# Define the time series of the EMG and ventilator recordings
y_emg = data_emg_samples[:, 1*fs_emg:61*fs_emg]
y_vent = data_vent_samples[:, 1*fs_vent:61*fs_vent]

# Define the time axes
t_emg = [i/fs_emg for i in range(len(y_emg[0, :]))]
t_vent = [i/fs_vent for i in range(len(y_vent[0, :]))]

In [ ]:
# Store the EMG data in a group of TimeSeries objects
emg_timeseries = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])

# Store the ventilator data in a group of TimeSeries objects
vent_timeseries = VentilatorDataGroup(
    y_vent,
    fs=fs_vent,
    labels=['Paw', 'F', 'Vvent'],
    units=['cmH2O', 'L/s', 'L'])


In [10]:
x_lim = 8.0
y_lim = 3.5

In [11]:
vent_timeseries.baseline(
    channel_idxs=[0],
    signal_type='raw')

# Find tidal volume peaks
p_vent = vent_timeseries.channels[vent_timeseries.p_vent_idx]
v_vent = vent_timeseries.channels[vent_timeseries.v_vent_idx]
vent_timeseries.find_tidal_volume_peaks(
    start_idx=0,
    end_idx=int(x_lim*fs_vent),
)
p_vent.peaks['ventilator_breaths'].detect_on_offset(
    baseline=p_vent.y_baseline)
v_vent.peaks['ventilator_breaths'].detect_on_offset(
    baseline=v_vent.y_baseline)

# 3a. Under-smoothed

In [ ]:
emg_timeseries_under = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])

emg_timeseries_under.filter(
    order=1,
    hp_cf=15,
    lp_cf=1000,

)
emg_timeseries_under.gating(
    fill_method=3,
    gate_width_samples=(150 *  fs_emg) // 1000
)
emg_timeseries_under.envelope(
    env_type='rms',
    env_window=(100 * fs_emg) // 1000,
)
emg_timeseries_under.baseline(
    percentile=20,
    window_s=int(7.5 * fs_emg)
)
emg_di_under = emg_timeseries_under.channels[1]
emg_di_under.detect_emg_breaths(
    peak_set_name='breaths',
    start_idx=0,
    end_idx = int(x_lim * fs_emg),
    min_peak_width_s=fs_emg//6,
    prominence_factor=0.5,
)
emg_di_under.peaks['breaths'].detect_on_offset(
    baseline=emg_di_under.y_baseline
)
t_pocc_peaks_vent = \
    p_vent.peaks['ventilator_breaths'].peak_df['peak_idx'].to_numpy() / p_vent.fs

emg_di_under.link_peak_set(
    peak_set_name='breaths',
    t_reference_peaks=t_pocc_peaks_vent,
    linked_peak_set_name='linked_breaths',
)
emg_di_under.peaks['breaths'].peak_df, emg_di_under.peaks['linked_breaths'].peak_df
# emg_di_under.peaks['breaths'].peak_df

In [ ]:
emg_timeseries_good = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])

emg_timeseries_good.filter(
    order=3,
    hp_cf=20,
    lp_cf=1000,

)
emg_timeseries_good.gating(
    fill_method=3,
    gate_width_samples=(200 *  fs_emg) // 1000
)
emg_timeseries_good.envelope(
    env_type='rms',
    env_window=(250 * fs_emg) // 1000,
)
emg_timeseries_good.baseline(
    percentile=33,
    window_s=int(7.5 * fs_emg)
)
emg_di_good = emg_timeseries_good.channels[1]
emg_di_good.detect_emg_breaths(
    peak_set_name='breaths',
    start_idx=0,
    end_idx = int(x_lim * fs_emg),
    min_peak_width_s=fs_emg//4,
    prominence_factor=1.5,
)
emg_di_good.peaks['breaths'].detect_on_offset(
    baseline=emg_di_good.y_baseline
)
t_pocc_peaks_vent = \
    p_vent.peaks['ventilator_breaths'].peak_df['peak_idx'].to_numpy() / p_vent.fs

emg_di_good.link_peak_set(
    peak_set_name='breaths',
    t_reference_peaks=t_pocc_peaks_vent,
    linked_peak_set_name='linked_breaths',
)
emg_di_good.peaks['breaths'].peak_df, emg_di_good.peaks['linked_breaths'].peak_df
# emg_di_under.peaks['breaths'].peak_df

In [ ]:
emg_timeseries_over = EmgDataGroup(
    y_emg,
    fs=fs_emg,
    labels=['ECG', 'EMGdi', 'EMGpara'],
    units=3*['uV'])

emg_timeseries_over.filter(
    order=3,
    hp_cf=30,
    lp_cf=500,

)
emg_timeseries_over.gating(
    fill_method=3,
    gate_width_samples=(300 *  fs_emg) // 1000
)
emg_timeseries_over.envelope(
    env_type='rms',
    env_window=(500 * fs_emg) // 1000,
)
emg_timeseries_over.baseline(
    percentile=33,
    window_s=int(7.5 * fs_emg)
)
emg_di_over = emg_timeseries_over.channels[1]
emg_di_over.detect_emg_breaths(
    peak_set_name='breaths',
    start_idx=0,
    end_idx = int(x_lim * fs_emg),
    min_peak_width_s=fs_emg//4,
    prominence_factor=1.5,
)
emg_di_over.peaks['breaths'].detect_on_offset(
    baseline=emg_di_over.y_baseline
)
t_pocc_peaks_vent = \
    p_vent.peaks['ventilator_breaths'].peak_df['peak_idx'].to_numpy() / p_vent.fs

emg_di_over.link_peak_set(
    peak_set_name='breaths',
    t_reference_peaks=t_pocc_peaks_vent,
    linked_peak_set_name='linked_breaths',
)
emg_di_over.peaks['breaths'].peak_df, emg_di_over.peaks['linked_breaths'].peak_df
# emg_di_under.peaks['breaths'].peak_df

In [ ]:
# Plot the identified Pocc peaks in p_vent and sEAdi
fig, axis = plt.subplots(nrows=3, ncols=2, figsize=(10, 6), sharex=True)
axes_emg = axis[:, 0]
colors = ['tab:cyan', 'tab:orange', 'tab:red']
emg_timeseries_under.plot_full(axes_emg[0], signal_type='env', channel_idxs=[1], colors=['tab:red', 'tab:orange'])
emg_di_under.plot_markers(peak_set_name='breaths', axes=axes_emg[0], colors=['k', 'k', 'k'])
axes_emg[0].set_ylabel('sEAdi (uV)')
axes_emg[0].set_ylim([0, y_lim])

emg_timeseries_good.plot_full(axes_emg[1], signal_type='env', channel_idxs=[1], colors=['tab:green', 'tab:olive'])
emg_di_good.plot_markers(peak_set_name='breaths', axes=axes_emg[1], colors=['k', 'k', 'k'])
axes_emg[1].set_ylabel('sEAdi (uV)')
axes_emg[1].set_ylim([0, y_lim])

emg_timeseries_over.plot_full(axes_emg[2], signal_type='env', channel_idxs=[1], colors=['tab:pink', 'tab:purple'])
emg_di_over.plot_markers(peak_set_name='breaths', axes=axes_emg[2], colors=['k', 'k', 'k'])
axes_emg[2].set_ylabel('sEAdi (uV)')
axes_emg[2].set_ylim([0, y_lim])

axes_emg[0].set_title('EMG data')
axes_emg[-1].set_xlabel('t (s)')

axes_vent = axis[:, 1]
vent_timeseries.plot_full(axes_vent)

axes_vent[0].set_title('Ventilator data')
axes_vent[-1].set_xlabel('t (s)')
axes_vent[-1].set_xlim([0, x_lim])
axes_emg[1].set_ylim([0, y_lim])

In [ ]:
# Plot the individual peaks
n_peaks = len(emg_di_under.peaks['breaths'].peak_df['start_idx'].to_numpy())
fig, axis = plt.subplots(nrows=2, ncols=n_peaks, figsize=(10, 6), sharey='row')

axes_emg = axis[0, :]
emg_di_under.plot_peaks(
    axes=axes_emg,
    peak_set_name='breaths', 
    margin_s=fs_emg,
    colors=['tab:red', 'tab:orange'])
emg_di_under.plot_markers(axes=axes_emg, peak_set_name='breaths', colors=['k', 'k', 'k'])
emg_di_good.plot_peaks(
    axes=axes_emg,
    peak_set_name='breaths', 
    margin_s=fs_emg,
    colors=['tab:green', 'tab:olive'])
emg_di_good.plot_markers(axes=axes_emg, peak_set_name='breaths', colors=['k', 'k', 'k'])
emg_di_over.plot_peaks(
    axes=axes_emg,
    peak_set_name='breaths', 
    margin_s=fs_emg,
    colors=['tab:pink', 'tab:purple'])
emg_di_over.plot_markers(axes=axes_emg, peak_set_name='breaths', colors=['k', 'k', 'k'])
axes_emg[0].set_ylabel('sEAdi (uV)')

axes_vent = axis[1, :]
p_vent.plot_peaks(axes=axes_vent, peak_set_name='ventilator_breaths')
p_vent.plot_markers(axes=axes_vent, peak_set_name='ventilator_breaths')

for axis in axes_vent:
    axis.set_xlabel('t (s)')

In [ ]:
# Calculate ETPdi & Test EMG quality
parameter_names = {
    'time_product': 'ETPdi'
}
# Under-filtered pipeline
emg_di_under.calculate_time_products(
    peak_set_name='breaths', parameter_name='ETPdi')
emg_di_under.calculate_time_products(
    peak_set_name='breaths', include_aub=False, parameter_name='ETPdi-aub')
emg_di_under.test_emg_quality(
    peak_set_name='breaths', parameter_names=parameter_names, verbose=False)

print(emg_di_under.peaks['breaths'].peak_df[['peak_idx', 'valid', 'ETPdi', 'AUB', 'ETPdi-aub']])
print(emg_di_under.peaks['breaths'].quality_values_df[['peak_idx', 'snr', 'aub', 'bell']])
print(emg_di_under.peaks['breaths'].quality_outcomes_df)
print('')

# Good pipeline
emg_di_good.calculate_time_products(
    peak_set_name='breaths', parameter_name='ETPdi')
emg_di_good.calculate_time_products(
    peak_set_name='breaths', include_aub=False, parameter_name='ETPdi-aub')
emg_di_good.test_emg_quality(
    peak_set_name='breaths', parameter_names=parameter_names, verbose=False)

print(emg_di_good.peaks['breaths'].peak_df[['peak_idx', 'valid', 'ETPdi', 'AUB', 'ETPdi-aub']])
print(emg_di_good.peaks['breaths'].quality_values_df[['peak_idx', 'snr', 'aub', 'bell']])
print(emg_di_good.peaks['breaths'].quality_outcomes_df)
print('')

# Over-filtered pipeline
emg_di_over.calculate_time_products(
    peak_set_name='breaths', parameter_name='ETPdi')
emg_di_over.calculate_time_products(
    peak_set_name='breaths', include_aub=False, parameter_name='ETPdi-aub')
emg_di_over.test_emg_quality(
    peak_set_name='breaths', parameter_names=parameter_names, verbose=False)

print(emg_di_over.peaks['breaths'].peak_df[['peak_idx', 'valid', 'ETPdi', 'AUB', 'ETPdi-aub']])
print(emg_di_over.peaks['breaths'].quality_values_df[['peak_idx', 'snr', 'aub', 'bell']])
print(emg_di_over.peaks['breaths'].quality_outcomes_df)
print('')